<h1>MADATORY PYTHON LIBRARIES</h1>

In [1]:
import folium #from conda prompt window: conda install folium
from folium import plugins
from owslib.wms import WebMapService #from conda prompt window: conda install OWSLib
from IPython.core.display import HTML
folium.__version__

u'0.8.2'

<H1>WEB MAP SERVICES</H1>

Web Map Services are available for almost all products within the Copernicus Marien Service.<br>
You can always check if a given product offers this service by exploring the 'services' tab in the Copernicus Marine Service Cataloge detailed view:

In [2]:
product_id = "GLOBAL_ANALYSIS_FORECAST_PHY_001_024"
detail_view = "http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id="+product_id
print(detail_view)

http://marine.copernicus.eu/services-portfolio/access-to-products/?option=com_csw&view=details&product_id=GLOBAL_ANALYSIS_FORECAST_PHY_001_024


Let's play with such service for getting a quick view of the variables of the above product

<h1>WMS - MapViews</h1>

<ul><li><h2>3D datasets</li></ul></h2>

Let's choose one of the above 3D datasets an display its variables; for example, the daily mean dataset:

In [97]:
dataset = 'global-analysis-forecast-phy-001-024'

In [98]:
url = 'http://nrt.cmems-du.eu/thredds/wms/'+dataset
wms = WebMapService(url)

In [99]:
for layer in wms.contents.keys():
    print(layer + ' = ' + wms[layer].title)

mlotst = ocean_mixed_layer_thickness_defined_by_sigma_theta
zos = sea_surface_height_above_geoid
bottomT = sea_water_potential_temperature_at_sea_floor
sithick = sea_ice_thickness
siconc = sea_ice_area_fraction
usi = eastward_sea_ice_velocity
vsi = northward_sea_ice_velocity
thetao = sea_water_potential_temperature
so = sea_water_salinity
uo = eastward_sea_water_velocity
vo = northward_sea_water_velocity
sea_ice_velocity = sea_ice_velocity
sea_water_velocity = sea_water_velocity


Let's choose one of the above variables and calculate the center of its bounding box:

In [100]:
target = 'thetao'

Let's zoom around the UK again and display the above layer:

In [101]:
legend = [wms[target].styles[key]['legend'] for key in wms[target].styles.keys()][0]
print(legend)

http://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-phy-001-024?REQUEST=GetLegendGraphic&LAYER=thetao&PALETTE=occam


In [102]:
m = folium.Map(
    location=[55.9533, 1.888],
    zoom_start=5,
    control_scale=True
)

style = [wms[target].styles[key]['title'] for key in wms[target].styles.keys()][0]
w = folium.raster_layers.WmsTileLayer(
    url=url,
    name=wms[target].title,
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
)
w.add_to(m)

legend_html = '<div style="position: fixed; bottom: 50px; right: 50px; z-index:9999"><img src="'+legend+'"></div>'
m.get_root().html.add_child(folium.Element(legend_html))
m

The questions know are 2: which time and depth am I seen?

Let's have a look to this layer time coverage:

In [103]:
ini = wms[target].timepositions[0].strip()
end = wms[target].timepositions[-1].strip()
time_coverage = (ini+'/'+end).split('/P')[0]
print(time_coverage)

2016-01-01T12:00:00.000Z/2019-03-15T12:00:00.000Z


Let's have a look to this layer depth coverage:

In [104]:
ini = wms[target].elevations[0].strip()
end = wms[target].elevations[-1].strip()
depth_coverage = (ini+'/'+end)
print(depth_coverage)

-0.49402499198913574/-5727.9169921875


Let's select explicity a time and depth:

In [106]:
m = folium.Map(
    location=[55.9533, 1.888],
    zoom_start=5,
    control_scale=True
)

style = [wms[target].styles[key]['title'] for key in wms[target].styles.keys()][0]
w = folium.raster_layers.WmsTileLayer(
    url=url,
    name=wms[target].title+' at '+wms[target].elevations[0].strip(),
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
    time = '2018-02-15T12:00:00.000Z',
    elevation = wms[target].elevations[0].strip(),
)
w2 = folium.raster_layers.WmsTileLayer(
    url=url,
    name=wms[target].title+' at '+wms[target].elevations[12].strip(),
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
    time = '2018-02-15T12:00:00.000Z',
    elevation = wms[target].elevations[12].strip(),
)
w.add_to(m)
w2.add_to(m)

folium.LayerControl().add_to(m)

legend_html = '<div style="position: fixed; bottom: 50px; right: 50px; z-index:9999"><img src="'+legend+'"></div>'
m.get_root().html.add_child(folium.Element(legend_html))

m

Let's reset the colorbar to see better the differences:

In [118]:
COLORSCALERANGE = '10.0,20.0'
legend = [wms[target].styles[key]['legend'] for key in wms[target].styles.keys()][0]+'&colorscalerange='+COLORSCALERANGE
print(legend)

http://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-phy-001-024?REQUEST=GetLegendGraphic&LAYER=thetao&PALETTE=occam&colorscalerange=10.0,20.0


In [122]:
m_dual = plugins.DualMap(location=[55.9533, 1.888], zoom_start=5, control_scale=True)

style = [wms[target].styles[key]['title'] for key in wms[target].styles.keys()][0]
w = folium.raster_layers.WmsTileLayer(
    url=url,
    name=wms[target].title+' at '+wms[target].elevations[0].strip(),
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
    time = '2018-08-15T12:00:00.000Z',
    elevation = wms[target].elevations[0].strip(),
    colorscalerange=COLORSCALERANGE
)
w2 = folium.raster_layers.WmsTileLayer(
    url=url,
    name=wms[target].title+' at '+wms[target].elevations[9].strip(),
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
    time = '2018-08-15T12:00:00.000Z',
    elevation = wms[target].elevations[9].strip(),
    colorscalerange=COLORSCALERANGE
)
w.add_to(m_dual.m1)
w2.add_to(m_dual.m2)

legend_html1 = '<div style="position: fixed; bottom: 50px; right:0px; z-index:9999"><img src="'+legend+'"></div>'
legend_html2 = '<div style="position: fixed; bottom: 50px; left:370px; z-index:9999"><img src="'+legend+'"></div>'
m_dual.get_root().html.add_child(folium.Element(legend_html1))
m_dual.get_root().html.add_child(folium.Element(legend_html2))

m_dual

Let's animate the target layer to see its whole evolution and not just the above time selections:

In [ ]:
w = folium.raster_layers.WmsTileLayer(
    url=url,
    name=wms[target].title+' at '+wms[target].elevations[0].strip(),
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=target,
    overlay=True,
    elevation = wms[target].elevations[0].strip(),
    colorscalerange=COLORSCALERANGE
)

In [ ]:
time_layer = plugins.TimestampedWmsTileLayers(
    w,
    period='PT1H',
    time_interval='2019-03-01T00:30:00.000Z/2019-03-04T23:30:00.000Z'
)

In [ ]:
m = folium.Map(
    location=[55.9533, 1.888],
    zoom_start=5,
    control_scale=True
)
tile_layer.add_to(m)
time_layer.add_to(m)
#layer style
legend_html = '<div style="position: fixed; bottom: 50px; right:0px; z-index:9999"><img src="'+legend+'"></div>'
#add legend to map
m.get_root().html.add_child(folium.Element(legend_html))
#show map
m